In [ ]:

from dataclasses import dataclass
import pandas as pd
from multicall import Call

from mainnet_launch.constants import (
    ALL_CHAINS,
    ROOT_PRICE_ORACLE,
    INCENTIVE_PRICING_STATS,
    LIQUIDATION_ROW,
    profile_function,
    ChainData,
    ETH_CHAIN,
)
from mainnet_launch.abis import DESTINATION_DEBT_REPORTING_SWAPPED_ABI
from mainnet_launch.data_fetching.get_events import fetch_events
from mainnet_launch.data_fetching.get_state_by_block import (
    get_raw_state_by_blocks,
    get_state_by_one_block,
    identity_with_bool_success,
    safe_normalize_with_bool_success,
)
from mainnet_launch.database.schema.postgres_operations import _exec_sql_and_cache
from mainnet_launch.database.schema.ensure_tables_are_current.using_onchain.update_transactions import (
    ensure_all_transactions_are_saved_in_db,
    insert_avoid_conflicts,
)


# emit Swapped(address(sellToken), address(buyToken), sellAmount, buyTokenAmountReceived, buyTokenAmountReceived);

# interface IAsyncSwapper {
#     error TokenAddressZero();
#     error SwapFailed();
#     error InsufficientBuyAmountReceived(uint256 buyTokenAmountReceived, uint256 buyAmount);
#     error InsufficientSellAmount();
#     error InsufficientBuyAmount();
#     error InsufficientBalance(uint256 balanceNeeded, uint256 balanceAvailable);

#     event Swapped(
#         address indexed sellTokenAddress,
#         address indexed buyTokenAddress,
#         uint256 sellAmount,
#         uint256 buyAmount,
#         uint256 buyTokenAmountReceived
#     );

#     /**
#      * @notice Swaps sellToken for buyToken
#      * @param swapParams Encoded swap data
#      * @return buyTokenAmountReceived The amount of buyToken received from the swap
#      */
#     function swap(
#         SwapParams memory swapParams
#     ) external payable returns (uint256 buyTokenAmountReceived);
# }


@dataclass
class VaultLiquidated:  
    tx_hash: str # primary keys
    log_index: int # primary keys

    destination_vault_address: str
    from_token_address: str
    to_token_address: str
    
    liquidated_amount:float # in terms of to_token_address


# swapped event

# event Swapped(
#     address indexed sellTokenAddress,
#     address indexed buyTokenAddress,
#     uint256 sellAmount,
#     uint256 buyAmount,
#     uint256 buyTokenAmountReceived
# );





# https://etherscan.io/tx/0x052b4231be3c2b28480b335085cad1c20ba838cccd5fc98e9c1d39e8502c9f11#eventlog

# VaultLiquidated (index_topic_1 address vault, index_topic_2 address fromToken, index_topic_3 address toToken, uint256 amount)View Source

# 133

# Topics
# 0 0x0272b5a6ff5cab190795f808ef35307240b8bc0011849cb8e15c093b40b22dfb
# 1: vault
# 0x0091Fec1B75013D1b83f4Bb82f0BEC4E256758CB # Tokemak-Dola USD Stablecoin-DOLA/sUSDe string
# 2: fromToken
# 0x4e3FBD56CD56c3e72c1403e103b45Db9da5B9D2B # CVX
# 3: toToken
# 0x865377367054516e17014CcdED1e7d814EDC9ce4 # dola
# Data


# amount :
# 583413391926390257

# at a vault level?


# https://etherscan.io/address/0xe2c7011866db4cc754f1b9b60b2f2999b5b54be4#code


# we also want reward added events



In [ ]:
# we want to track two things,

# claim vault rewards, https://sonicscan.org/tx/0x1a8bb472b282690e5f9fa66a7ab8ba3f9765435fac429f643a5c14172d62485a
# where we grab some of the incentive tokens


# some options?
# CurveGaugeDestinationVault.BalanceUpdated(token, vault)



# What we care about for incentive tokens

- How accurate is our selling of incentive tokens acheived and oracle token prices
- How frequent is our selling of incentive tokens (for the base asset)
- How fequently are we claiming tokens,

    - be fine with false positives

In [1]:
contract

NameError: name 'contract' is not defined

In [5]:
from mainnet_launch.constants import *
from mainnet_launch.data_fetching.get_events import fetch_events
from mainnet_launch.abis import DESTINATION_DEBT_REPORTING_SWAPPED_ABI


chain = ETH_CHAIN
contract = chain.client.eth.contract(LIQUIDATION_ROW(chain), abi=DESTINATION_DEBT_REPORTING_SWAPPED_ABI)
swapped_df = fetch_events(contract.events.Swapped, chain=chain, start_block=chain.block_autopool_first_deployed)

swapped_df


,event,block,transaction_index,log_index,hash,sellTokenAddress,buyTokenAddress,sellAmount,buyAmount,buyTokenAmountReceived
0,Swapped,20786622,89,212,0xeaff1c935812795d7c751554c06a6f725d0523fe985d...,0xC0c293ce456fF0ED870ADd98a0828Dd4d2903DBF,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,405885564600978683213,65660904437741059,65660904437741083
1,Swapped,20786622,89,239,0xeaff1c935812795d7c751554c06a6f725d0523fe985d...,0xba100000625a3754423978a60c9317c58a424e3D,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,97656862179630371841,73583617510509109,73583617510509114
2,Swapped,20786622,89,263,0xeaff1c935812795d7c751554c06a6f725d0523fe985d...,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,305110013,123151123466434536,123151125084164991
3,Swapped,20786622,89,283,0xeaff1c935812795d7c751554c06a6f725d0523fe985d...,0x4e3FBD56CD56c3e72c1403e103b45Db9da5B9D2B,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,6322527432225350368,5347173311905731,5347173311905731
4,Swapped,20786622,89,307,0xeaff1c935812795d7c751554c06a6f725d0523fe985d...,0xD533a949740bb3306d119CC777fa900bA034cd52,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,2107509144075116791221,241327419724972605,241327419724972627
...,...,...,...,...,...,...,...,...,...,...
2504,Swapped,22226764,1,42,0xecab7622fdb1c2796d3d29082fcbd03227b5212cc68d...,0xba100000625a3754423978a60c9317c58a424e3D,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,72257044740448401664,37371696149337135,37749188029633470
2505,Swapped,22226764,1,68,0xecab7622fdb1c2796d3d29082fcbd03227b5212cc68d...,0x6DF0E641FC9847c0c6Fde39bE6253045440c14d3,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,9485122782053310905487,101049647263052245,102070350770759844
2506,Swapped,22226764,1,80,0xecab7622fdb1c2796d3d29082fcbd03227b5212cc68d...,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,108384545,72631251566247817,73364900571967492
2507,Swapped,22226764,1,94,0xecab7622fdb1c2796d3d29082fcbd03227b5212cc68d...,0x4e3FBD56CD56c3e72c1403e103b45Db9da5B9D2B,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,4379950906421052627,4997222341514878,5047699334863514
